In [12]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import mannwhitneyu

In [80]:
stat_df = pd.read_csv('experiment_beta_stat_data.csv')
stat_df.head()

,beta,coefficient,S.E. of beta,pval of beta,quantile,model,method,trial
0,beta1,0.120323,0.075142,5.465903e-02,0.05,igarch,numeric,0
1,beta2,0.715896,0.066497,2.495006e-27,0.05,igarch,numeric,0
2,beta3,0.723071,0.091509,1.375906e-15,0.05,igarch,numeric,0
3,beta1,0.119824,0.074168,5.309481e-02,0.05,igarch,numeric,1
4,beta2,0.715070,0.065989,1.158424e-27,0.05,igarch,numeric,1


In [91]:
stat_df['pval is pass'] = stat_df['pval of beta'] < 0.05

In [93]:
stat_df.groupby(['quantile', 'model', 'beta', 'method'])['pval is pass'].value_counts()

quantile  model       beta   method   pval is pass
0.01      adaptive    beta1  mle      True            30
                             numeric  True            30
          asymmetric  beta1  mle      True            24
                                      False            6
                             numeric  True            27
                                      False            3
                      beta2  mle      True            30
                             numeric  True            30
                      beta3  mle      False           30
                             numeric  False           30
                      beta4  mle      True            26
                                      False            4
                             numeric  True            27
                                      False            3
          igarch      beta1  mle      False           30
                             numeric  False           30
                      beta2  mle     

In [86]:
for metric in ['coefficient', 'pval of beta']:
    for q in stat_df['quantile'].unique():
        for mod in stat_df['model'].unique():
            cond1 = stat_df['quantile']==q
            cond2 = stat_df['model']==mod
            x = stat_df[(stat_df['method']=='mle')&cond1&cond2][metric]
            y = stat_df[(stat_df['method']=='numeric')&cond1&cond2][metric]
            print(mod, q)
            print(metric, np.round(x.mean(), 4), np.round(y.mean(), 4))
            print('accepted' if mannwhitneyu(x, y)[1] > 0.05 else 'rejected')

igarch 0.05
coefficient 0.5213 0.5199
accepted
symmetric 0.05
coefficient 0.0381 0.0352
accepted
asymmetric 0.05
coefficient 0.3385 0.3382
accepted
adaptive 0.05
coefficient -0.9661 -0.9224
accepted
igarch 0.01
coefficient 0.9203 0.8984
accepted
symmetric 0.01
coefficient -0.1492 -0.1609
accepted
asymmetric 0.01
coefficient 0.2485 0.2543
accepted
adaptive 0.01
coefficient -0.8219 -0.8767
accepted
igarch 0.05
pval of beta 0.0172 0.018
accepted
symmetric 0.05
pval of beta 0.0029 0.003
accepted
asymmetric 0.05
pval of beta 0.0989 0.1009
accepted
adaptive 0.05
pval of beta 0.0 0.0
accepted
igarch 0.01
pval of beta 0.0262 0.042
rejected
symmetric 0.01
pval of beta 0.0149 0.0226
accepted
asymmetric 0.01
pval of beta 0.097 0.0945
accepted
adaptive 0.01
pval of beta 0.0001 0.0001
accepted


In [5]:
stat_df = pd.read_csv('experiment_stat_data.csv')
stat_df.head()

,quantile,model,method,trial,loss,hit_rate_in,hit_rate_out,dq_in,dq_out,binom_in,binom_out,traffic_in,traffic_out,kupiec_in,kupiec_out,independent_in,independent_out
0,0.05,igarch,numeric,0,0.116262,0.049915,0.069272,0.244899,0.970888,1.000000,0.041729,green,yellow,1.0,1.0,0.775363,0.186185
1,0.05,igarch,numeric,1,0.116262,0.049348,0.069272,0.329163,0.970943,0.956417,0.041729,green,yellow,1.0,1.0,0.795643,0.186185
2,0.05,igarch,numeric,2,0.116262,0.049915,0.069272,0.244856,0.970896,1.000000,0.041729,green,yellow,1.0,1.0,0.775363,0.186185
3,0.05,igarch,numeric,3,0.116262,0.050482,0.069272,0.214165,0.970905,0.912938,0.041729,green,yellow,1.0,1.0,0.753776,0.186185
4,0.05,igarch,numeric,4,0.116261,0.049348,0.069272,0.279176,0.970883,0.956417,0.041729,green,yellow,1.0,1.0,0.795643,0.186185


In [75]:
stat_to_csv_df = {
    'model': [],
    'quantile': [],
    'loss': [],
    'mle\'s loss': [],
    'numeric\'s loss': [],
    'hit_rate_in': [],
    'mle\'s hit_rate_in': [],
    'numeric\'s hit_rate_in': [],
    'hit_rate_out': [],
    'mle\'s hit_rate_out': [],
    'numeric\'s hit_rate_out': [],
}

keys = [key for key in stat_to_csv_df][2:]

for q in stat_df['quantile'].unique():
    for mod in stat_df['model'].unique():
        cond1 = stat_df['quantile']==q
        cond2 = stat_df['model']==mod
        
        stat_to_csv_df['model'].append(mod)
        stat_to_csv_df['quantile'].append(q)
        
        for i, metric in enumerate(stat_df.columns[4:7]):
            x = stat_df[(stat_df['method']=='mle')&cond1&cond2][metric]
            y = stat_df[(stat_df['method']=='numeric')&cond1&cond2][metric]
            stat_to_csv_df[keys[i*3]].append('accepted' if mannwhitneyu(x, y)[1] > 0.05 else 'rejected')
            stat_to_csv_df[keys[i*3+1]].append(np.round(x.mean(), 4))
            stat_to_csv_df[keys[i*3+2]].append(np.round(y.mean(), 4))

In [79]:
pd.DataFrame(stat_to_csv_df).T

,0,1,2,3,4,5,6,7
model,igarch,symmetric,asymmetric,adaptive,igarch,symmetric,asymmetric,adaptive
quantile,0.05,0.05,0.05,0.05,0.01,0.01,0.01,0.01
loss,accepted,rejected,accepted,rejected,rejected,accepted,rejected,accepted
mle's loss,0.1163,0.1176,0.1098,0.1266,0.0354,0.0351,0.0336,0.0479
numeric's loss,0.1163,0.1176,0.1098,0.1264,0.0354,0.0351,0.0335,0.0479
hit_rate_in,accepted,rejected,accepted,accepted,rejected,accepted,accepted,accepted
mle's hit_rate_in,0.05,0.0504,0.0504,0.0455,0.0106,0.0101,0.01,0.0099
numeric's hit_rate_in,0.05,0.0501,0.0506,0.0453,0.0109,0.0101,0.01,0.0099
hit_rate_out,accepted,accepted,accepted,rejected,accepted,accepted,accepted,accepted
mle's hit_rate_out,0.0694,0.0765,0.0782,0.0471,0.0073,0.007,0.0134,0.0101
